In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys, os
sys.path.append(r'C:\git\gmt-fom-gas-dashboard')
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')
sys.path.append(r'C:\git\gmt-fom-power-utils')

In [ ]:
import smartdata
from smartdata.utils.common import print_attributes as pa
from smartdata.utils.dframe import display_full_df as ddff
from smartdata.utils.common import to_namedtuple

In [ ]:
from datetime import datetime
import pandas as pd
import requests

In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table

from datetime import datetime
from dateutil.relativedelta import relativedelta
import urllib.parse

import numpy as np
import pandas as pd

In [ ]:
%set_env FLASK_APP=C:\dev\code\gmt-amp\python-analytics-platform\gmt\dna\amp\api\runner.py

import sys,os,os.path
os.environ['PYTHONUNBUFFERED'] = '1'
os.environ['AMP_API_CONFIG'] = r'C:\dev\code\gmt-amp\python-analytics-platform\config\api.dev.yml'
os.environ['USE_WERKZEUG'] = '1'

In [ ]:
al = 0

In [ ]:
amp_df = pd.DataFrame(index=['No AMP data available']).reset_index()
amp_df.columns = ['']
amp_df.index = pd.DatetimeIndex([datetime.today().date()])
amp_df.index.name = 'PointDate'
amp_df.columns = ['curve']

In [ ]:
kk = amp_df.to_json(date_format='iso', orient='split')

In [ ]:
'No AMP data available' in kk

In [ ]:
pd.read_json(kk, orient='split')

# Retrieve from AMP

In [ ]:
from gmt.dna.amp.api.app import app, configure_app

In [ ]:
with app.app_context() as ctx:
    configure_app()

In [ ]:
config = app.config

In [ ]:
first_model_run_id = int(15838)

In [ ]:
view_from_date = datetime(2020,9,3)
view_to_date = datetime(2022,9,3)

In [ ]:
view_from_date = config['TIMEZONE'].localize(view_from_date)
view_to_date = config['TIMEZONE'].localize(view_to_date)

In [ ]:
from gmt.dna.amp.api.services import data

In [ ]:
from gmt.dna.amp.api.entities.symbol import get_symbol_names

In [ ]:
from gmt.dna.amp.api.dal.db import load_model_run_from_db

In [ ]:
from gmt.dna.amp.api.services.model_run import get_model_run
from gmt.dna.amp.api.services.model import load_model

In [ ]:
with app.app_context() as ctx:
    configure_app()
    model_run = get_model_run(model_run_id=first_model_run_id, username=None)
    model = load_model(model_run.model_name)

In [ ]:
max_group_level = 1
sheets = None

In [ ]:
from gmt.dna.amp.common.constants import SymbolTypes

In [ ]:
symbol_dict = {
        symbol.symbol_name: symbol
        for symbol in model.get_displayed_symbols(max_group_level=max_group_level, sheets=sheets)
        # Consider only the time series symbols
        if symbol.symbol_type in {SymbolTypes.Actual, SymbolTypes.Curve, SymbolTypes.Forecast}
    }

In [ ]:
from gmt.dna.amp.api.entities.symbol import get_symbol_names

In [ ]:
symbols = list(symbol_dict.values())
symbol_names = get_symbol_names(symbols=symbols)

In [ ]:
from gmt.dna.amp.api.utils import convert_datetime
from pytz import utc

In [ ]:
view_from_date_utc = convert_datetime(dt=view_from_date, dst_tz=utc)
view_to_date_utc = convert_datetime(dt=view_to_date, dst_tz=utc)

In [ ]:
with app.app_context() as ctx:
    first_data_result = data.get_all_symbol_data(first_model_run_id, symbol_names, view_from_date_utc, view_to_date_utc)

In [ ]:
res_df = pd.DataFrame()
for itm in first_data_result.items:
    print(f'Working {itm.name}')
    df = pd.DataFrame(itm.data)    
    df.columns = ['PointDate', itm.name]
    df.set_index('PointDate', inplace=True)
    res_df = pd.concat([res_df, df], axis=1)

In [ ]:
_ = res_df.iloc[:,:1]

In [ ]:
k = _.to_json(date_format='iso', orient='split')

In [ ]:
()

In [ ]:
pd.read_json(k, orient='split')

In [ ]:
res_df.fillna(0).iloc[:,:7].plot(figsize=(8*1.6,8))

## Retrieve from AMP (class)

In [72]:
from gmt.dna.amp.api.services import arc_data

from_date = view_from_date_utc
to_date = view_to_date_utc

arc_amp_calc = arc_data.ArcAmpCalc(model=model.name)

In [74]:
with app.app_context() as ctx:
    res_df = arc_amp_calc.get_amp_curves(first_model_run_id, from_date, to_date)

2020-09-08 07:39:37,496 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.data: Loading 197 symbols out of 197 for model run 15838: NL.SUPPLY.GRONINGEN/GAS/FLOW/CURVE/SOD, NL.STORAGE.NORG-OFFGRID/GAS/FLOW/CURVE/SOD, NL.SUPPLY.GTS.PRODUCTION/GAS/FLOW/CURVE/SOD, NL.SUPPLY.NORWAY/GAS/FLOW/CURVE/SOD, NL.SUPPLY.LNG/GAS/FLOW/CURVE/SOD, NL.SUPPLY.TOTAL/GAS/FLOW/CURVE/SOD, NL.DEMAND.LDZ/GAS/FLOW/CURVE/SOD, NL.DEMAND.INDUSTRY/GAS/FLOW/CURVE/SOD, NL.DEMAND.TOTAL/GAS/FLOW/CURVE/SOD, NL.BORDERFLOW.GERMANY.H/GAS/FLOW/CURVE/SOD ...
2020-09-08 07:40:02,840 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.data: Loaded 197 symbols


2020-09-08 07:40:02,842 - gmt.dna.amp.api.services.arc_data - INFO - Retrieving amp curve: NL.SUPPLY.GRONINGEN/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:02,842 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.arc_data: Retrieving amp curve: NL.SUPPLY.GRONINGEN/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:02,846 - gmt.dna.amp.api.services.arc_data - INFO - Retrieving amp curve: NL.STORAGE.NORG-OFFGRID/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:02,846 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.arc_data: Retrieving amp curve: NL.STORAGE.NORG-OFFGRID/GAS/FLOW/CURVE/SOD


...

2020-09-08 07:40:03,682 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.arc_data: Retrieving amp curve: NL.STORAGE.VNG.SSO.NET.WITHDRAWALS/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:03,688 - gmt.dna.amp.api.services.arc_data - INFO - Retrieving amp curve: NL.STORAGE.GAS.UNION.SSO.PERCENTAGE/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:03,688 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.arc_data: Retrieving amp curve: NL.STORAGE.GAS.UNION.SSO.PERCENTAGE/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:03,692 - gmt.dna.amp.api.services.arc_data - INFO - Retrieving amp curve: NL.STORAGE.GAS.UNION.SSO.NET.WITHDRAWALS/GAS/FLOW/CURVE/SOD


2020-09-08 07:40:03,692 [INFO] [NO-REQ] [NO-USER] gmt.dna.amp.api.services.arc_data: Retrieving amp curve: NL.STORAGE.GAS.UNION.SSO.NET.WITHDRAWALS/GAS/FLOW/CURVE/SOD


In [75]:
res_df.head()

NL.SUPPLY.GRONINGEN/GAS/FLOW/CURVE/SOD  \
PointDate                                                           
2020-09-02 22:00:00+00:00                               15.305425   
2020-09-03 22:00:00+00:00                               18.000000   
2020-09-04 22:00:00+00:00                               18.000000   
2020-09-05 22:00:00+00:00                               18.000000   
2020-09-06 22:00:00+00:00                               18.000000   

                           NL.STORAGE.NORG-OFFGRID/GAS/FLOW/CURVE/SOD  \
PointDate                                                               
2020-09-02 22:00:00+00:00                                   -0.361393   
2020-09-03 22:00:00+00:00                                    0.320000   
2020-09-04 22:00:00+00:00                                    0.320000   
2020-09-05 22:00:00+00:00                                    0.320000   
2020-09-06 22:00:00+00:00                                    0.320000   

                           NL.SUPPLY.GTS.PRODUCTION/GAS/FLOW/CURVE/SOD  \
PointDate                                                                
2020-09-02 22:00:00+00:00                                    42.816637   
2020-09-03 22:00:00+00:00                                    46.192606   
2020-09-04 22:00:00+00:00                                    46.192606   
2020-09-05 22:00:00+00:00                                    46.192606   
2020-09-06 22:00:00+00:00                                    46.192606   

                           NL.SUPPLY.NORWAY/GAS/FLOW/CURVE/SOD  \
PointDate                                                        
2020-09-02 22:00:00+00:00                            74.276413   
2020-09-03 22:00:00+00:00                            76.823986   
2020-09-04 22:00:00+00:00                            76.050721   
2020-09-05 22:00:00+00:00                            40.000000   
2020-09-06 22:00:00+00:00                            40.000000   

                           NL.SUPPLY.LNG/GAS/FLOW/CURVE/SOD  \
PointDate                                                     
2020-09-02 22:00:00+00:00                         18.642702   
2020-09-03 22:00:00+00:00                          9.663586   
2020-09-04 22:00:00+00:00                          3.981212   
2020-09-05 22:00:00+00:00                         13.500000   
2020-09-06 22:00:00+00:00                         19.800000   

                           NL.SUPPLY.TOTAL/GAS/FLOW/CURVE/SOD  \
PointDate                                                       
2020-09-02 22:00:00+00:00                          135.735752   
2020-09-03 22:00:00+00:00                          132.680178   
2020-09-04 22:00:00+00:00                          126.224538   
2020-09-05 22:00:00+00:00                           99.692606   
2020-09-06 22:00:00+00:00                          105.992606   

                           NL.DEMAND.LDZ/GAS/FLOW/CURVE/SOD  \
PointDate                                                     
2020-09-02 22:00:00+00:00                        -23.279388   
2020-09-03 22:00:00+00:00                        -19.574596   
2020-09-04 22:00:00+00:00                        -17.178375   
2020-09-05 22:00:00+00:00                        -19.280530   
2020-09-06 22:00:00+00:00                        -23.666346   

                           NL.DEMAND.INDUSTRY/GAS/FLOW/CURVE/SOD  \
PointDate                                                          
2020-09-02 22:00:00+00:00                             -54.061474   
2020-09-03 22:00:00+00:00                             -54.556357   
2020-09-04 22:00:00+00:00                             -50.930000   
2020-09-05 22:00:00+00:00                             -50.930000   
2020-09-06 22:00:00+00:00                             -59.410000   

                           NL.DEMAND.TOTAL/GAS/FLOW/CURVE/SOD  \
PointDate                                                       
2020-09-02 22:00:00+00:00                          -77.340863   
2020-09-03 22:00:00+00:00            

In [ ]:
pd.read

In [ ]:
pd.read_json(res_df.to_json(date_format='iso', orient='split'), orient='split')

In [ ]:
pd.DataFrame(res_df.head().to_dict(orient='records'))